In [119]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.finance import candlestick_ohlc
import matplotlib.dates as mdates
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

%matplotlib inline
plt.rcParams["figure.figsize"] = (14, 10)
plt.rcParams["font.size"] = 14

In [95]:
df_train = pd.read_csv("train.csv", index_col=0)
df_test = pd.read_csv("test.csv", index_col=0)
#df_train['Date'] = pd.to_datetime(df_train['Date'])
#df_test["Date"] = pd.to_datetime(df_test['Date'])

In [96]:
#df_train['Date'] = pd.to_datetime(df_train['Date'])
#df_test["Date"] = pd.to_datetime(df_test['Date'])
df_train.head()

,Open Price,Close Price,High Price,Low Price,Volume
Date,,,,,
02-Jan-2009,902.99,931.80,934.73,899.35,4048270080
05-Jan-2009,929.17,927.45,936.63,919.53,5413910016
06-Jan-2009,931.17,934.70,943.85,927.28,5392620032
07-Jan-2009,927.45,906.65,927.45,902.37,4704940032
08-Jan-2009,905.73,909.73,910.00,896.81,4991549952


## Check if any null data in the dataset

In [97]:
df_train.isnull().sum()

Open Price     0
Close Price    0
High Price     0
Low Price      0
Volume         0
dtype: int64

## preprocessing
calculate the difference between the prices, and calculate the price is higher or lower than the day before that day

In [98]:
df_train["return"] = df_train["High Price"] - df_train["Low Price"]
df_train["close to open"] = np.abs(df_train["Close Price"] - df_train["Open Price"])
df_train["close to high"] = np.abs(df_train["Close Price"] - df_train["High Price"])
df_train["close to low"] = np.abs(df_train["Close Price"] - df_train["Low Price"])

df_test["return"] = df_test["High Price"] - df_test["Low Price"]
df_test["close to open"] = np.abs(df_test["Close Price"] - df_test["Open Price"])
df_test["close to high"] = np.abs(df_test["Close Price"] - df_test["High Price"])
df_test["close to low"] = np.abs(df_test["Close Price"] - df_test["Low Price"])

def result_calc(x):
    if x > 0:
        return 1
    elif x < 0:
        return 0
    elif x == 0:
        return 2

df_train["result"] = df_train["Close Price"] - df_train["Close Price"].shift(1)
df_train["result"] = df_train["result"].apply(lambda x: result_calc(x))

df_test["result"] = df_test["Close Price"] - df_test["Close Price"].shift(1)
df_test["result"] = df_test["result"].apply(lambda x: result_calc(x))

df_train.head()

,Open Price,Close Price,High Price,Low Price,Volume,return,close to open,close to high,close to low,result
Date,,,,,,,,,,
02-Jan-2009,902.99,931.80,934.73,899.35,4048270080,35.38,28.81,2.93,32.45,NaN
05-Jan-2009,929.17,927.45,936.63,919.53,5413910016,17.10,1.72,9.18,7.92,0.0
06-Jan-2009,931.17,934.70,943.85,927.28,5392620032,16.57,3.53,9.15,7.42,1.0
07-Jan-2009,927.45,906.65,927.45,902.37,4704940032,25.08,20.80,20.80,4.28,0.0
08-Jan-2009,905.73,909.73,910.00,896.81,4991549952,13.19,4.00,0.27,12.92,1.0


### Because the result of first row is null, drop it

In [99]:
df_train["result"].isnull().sum()

1

In [100]:
df_train = df_train.dropna()
df_test = df_test.dropna()
df_train.head()

,Open Price,Close Price,High Price,Low Price,Volume,return,close to open,close to high,close to low,result
Date,,,,,,,,,,
05-Jan-2009,929.17,927.45,936.63,919.53,5413910016,17.10,1.72,9.18,7.92,0.0
06-Jan-2009,931.17,934.70,943.85,927.28,5392620032,16.57,3.53,9.15,7.42,1.0
07-Jan-2009,927.45,906.65,927.45,902.37,4704940032,25.08,20.80,20.80,4.28,0.0
08-Jan-2009,905.73,909.73,910.00,896.81,4991549952,13.19,4.00,0.27,12.92,1.0
09-Jan-2009,909.91,890.35,911.93,888.31,4716499968,23.62,19.56,21.58,2.04,0.0


## Normalize the elements

In [101]:
scaler = StandardScaler()
scaler.fit(df_train[["Open Price", "Close Price", "High Price", "Low Price", "Volume", "return", "close to open", "close to high", "close to low"]])
df_train[["Open Price", "Close Price", "High Price", "Low Price", "Volume", "return", "close to open", "close to high", "close to low"]] = scaler.transform(df_train[["Open Price", "Close Price", "High Price", "Low Price", "Volume", "return", "close to open", "close to high", "close to low"]])
df_train.head()


,Open Price,Close Price,High Price,Low Price,Volume,return,close to open,close to high,close to low,result
Date,,,,,,,,,,
05-Jan-2009,-1.499495,-1.504450,-1.502622,-1.500502,1.824404,0.010751,-0.836174,0.168867,-0.171102,0.0
06-Jan-2009,-1.495371,-1.489501,-1.487722,-1.484533,1.808651,-0.042903,-0.644738,0.165531,-0.231612,1.0
07-Jan-2009,-1.503041,-1.547337,-1.521567,-1.535860,1.299815,0.818598,1.181833,1.461159,-0.611611,0.0
08-Jan-2009,-1.547821,-1.540987,-1.557579,-1.547317,1.511887,-0.385074,-0.595028,-0.822038,0.433992,1.0
09-Jan-2009,-1.539203,-1.580946,-1.553596,-1.564831,1.308369,0.670796,1.050684,1.547905,-0.882693,0.0


In [102]:
scaler.fit(df_test[["Open Price", "Close Price", "High Price", "Low Price", "Volume", "return", "close to open", "close to high", "close to low"]])
df_test[["Open Price", "Close Price", "High Price", "Low Price", "Volume", "return", "close to open", "close to high", "close to low"]] = scaler.transform(df_test[["Open Price", "Close Price", "High Price", "Low Price", "Volume", "return", "close to open", "close to high", "close to low"]])

df_test.head()

,Open Price,Close Price,High Price,Low Price,Volume,return,close to open,close to high,close to low,result
Date,,,,,,,,,,
03-Jan-2018,-0.508581,-0.331044,-0.523646,-0.305917,-0.142641,-0.689624,-0.135579,-0.753110,-0.035561,1.0
04-Jan-2018,-0.291669,-0.221964,-0.363034,-0.105820,-0.120207,-0.963246,-0.686423,-0.558909,-0.643437,1.0
05-Jan-2018,-0.170175,-0.030748,-0.210605,-0.022681,-0.521355,-0.735513,-0.312916,-0.802268,-0.039078,1.0
08-Jan-2018,-0.055553,0.014761,-0.156135,0.068256,-0.574383,-0.933653,-0.667590,-0.777933,-0.339206,1.0
09-Jan-2018,0.030160,0.050489,-0.041704,0.164641,-0.436682,-0.917785,-0.923918,-0.434795,-0.730779,1.0


## GridSearchCV test

In [103]:
y = df_train["result"]
y_test = df_test["result"]
df_train = df_train.drop("result", 1)
df_test = df_test.drop("result", 1)

df_train.head()

,Open Price,Close Price,High Price,Low Price,Volume,return,close to open,close to high,close to low
Date,,,,,,,,,
05-Jan-2009,-1.499495,-1.504450,-1.502622,-1.500502,1.824404,0.010751,-0.836174,0.168867,-0.171102
06-Jan-2009,-1.495371,-1.489501,-1.487722,-1.484533,1.808651,-0.042903,-0.644738,0.165531,-0.231612
07-Jan-2009,-1.503041,-1.547337,-1.521567,-1.535860,1.299815,0.818598,1.181833,1.461159,-0.611611
08-Jan-2009,-1.547821,-1.540987,-1.557579,-1.547317,1.511887,-0.385074,-0.595028,-0.822038,0.433992
09-Jan-2009,-1.539203,-1.580946,-1.553596,-1.564831,1.308369,0.670796,1.050684,1.547905,-0.882693


In [143]:
param_grid = {"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}

grid = GridSearchCV(LogisticRegression(), param_grid=param_grid)
grid.fit(df_train, y)
grid.predict(df_test)
print(grid.best_score_)
print(grid.best_params_)

0.9072028281042863
{'C': 10.0, 'penalty': 'l1'}


C = 10, penalty = 11

In [150]:
log = LogisticRegression(C=10)
log.fit(df_train, y)
y_predict = log.predict(df_test)


In [151]:
accuracy_score(y_predict, y_test)

0.8326693227091634

## accuracy test with train_test_split

In [153]:
x1, x2, y1, y2 = train_test_split(df_train, y, train_size=0.7)
a = LogisticRegression(C=10.0)
a.fit(x1, y1)
yp = a.predict(x2)
accuracy_score(yp, y2)

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning:

From version 0.21, test_size will always complement train_size unless both are specified.



0.8409425625920471